In [1]:
import pandas as pd
import numpy as np

In [2]:
bra = pd.read_excel('BRA2020.xlsx')
bra

,Temporada,Rodada,Mandante,Score_m,Visitante,Score_v
0,2020,1,FOR,0,CAP,2
1,2020,1,CFC,0,INT,1
2,2020,1,SPT,3,CEA,2
3,2020,1,FLA,0,CAM,1
4,2020,1,SAN,1,BGT,1
...,...,...,...,...,...,...
375,2020,38,INT,0,COR,0
376,2020,38,BAH,2,SAN,0
377,2020,38,CAP,2,SPT,0
378,2020,38,CEA,2,BOT,1


In [3]:
bra = bra[bra['Score_m'].notnull()]
bra

,Temporada,Rodada,Mandante,Score_m,Visitante,Score_v
0,2020,1,FOR,0,CAP,2
1,2020,1,CFC,0,INT,1
2,2020,1,SPT,3,CEA,2
3,2020,1,FLA,0,CAM,1
4,2020,1,SAN,1,BGT,1
...,...,...,...,...,...,...
375,2020,38,INT,0,COR,0
376,2020,38,BAH,2,SAN,0
377,2020,38,CAP,2,SPT,0
378,2020,38,CEA,2,BOT,1


In [22]:
times = bra['Mandante'].unique()
tabela = pd.DataFrame()
for time in times:
    vit = emp = der = pro = con = jog = 0
    # emp = 0
    # der = 0
    # pro = 0
    # con = 0
    # jog = 0
    mandante = bra[bra['Mandante'] == time]
    for indice, partida in mandante.iterrows():
        if partida['Score_m'] > partida['Score_v']:
            vit += 1 
        elif partida['Score_m'] == partida['Score_v']:
            emp += 1
        else:
            der += 1
        pro += partida['Score_m']
        con += partida['Score_v']
        jog += 1
    visitante = bra[bra['Visitante'] == time]
    for indice, partida in visitante.iterrows():
        if partida['Score_v'] > partida['Score_m']:
            vit += 1
        elif partida['Score_v'] == partida['Score_m']:
            emp += 1
        else:
            der += 1
        pro += partida['Score_v']
        con += partida['Score_m']
        jog += 1
    new_row = pd.DataFrame({
        '1-Time': [time],
        '3-J': [jog],
        '4-V': [vit],
        '5-E' : [emp],
        '6-D': [der],
        '7-GP': [pro],
        '8-GC': [con]
    })   
    tabela = pd.concat([tabela, new_row], ignore_index = True)
    
tabela['2-P'] = (tabela['4-V'] * 3) + tabela['5-E']
tabela['9-SG'] = tabela['7-GP'] - tabela['8-GC']   
tabela_sort = tabela.sort_values(by=['2-P','4-V','9-SG'], ascending=False)
tabela_sort

,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG
3,FLA,38,21,8,9,68,48,71,20
18,INT,38,20,10,8,61,35,70,26
11,CAM,38,20,8,10,63,44,68,19
17,SAO,38,18,12,8,59,41,66,18
15,FLU,38,18,10,10,55,42,64,13
5,GRE,38,14,17,7,53,40,59,13
9,PAL,38,15,13,10,51,37,58,14
4,SAN,38,14,12,12,52,51,54,1
12,CAP,38,15,8,15,38,36,53,2
10,BGT,38,13,14,11,50,40,53,10


,A,B
0,1,3
1,2,4
0,5,7
1,6,8


In [23]:
tabela

,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG
0,FOR,38,10,11,17,34,44,41,-10
1,CFC,38,7,10,21,31,54,31,-23
2,SPT,38,12,6,20,31,50,42,-19
3,FLA,38,21,8,9,68,48,71,20
4,SAN,38,14,12,12,52,51,54,1
5,GRE,38,14,17,7,53,40,59,13
6,BOT,38,5,12,21,31,61,27,-30
7,COR,38,13,12,13,45,45,51,0
8,GOI,38,9,10,19,41,63,37,-22
9,PAL,38,15,13,10,51,37,58,14


In [24]:
from sklearn.cluster import KMeans
df_data = tabela[[
    '2-P',              
    '4-V',
    '5-E',
    '6-D',
    '9-SG'
]]
kmeans = KMeans(n_clusters=5, random_state=0).fit(df_data)
tabela['cluster'] = kmeans.labels_
# tabela['c_p'] = kmeans.cluster_centers_[kmeans.labels_,0]
# tabela['c_v'] = kmeans.cluster_centers_[kmeans.labels_,1]
# tabela['c_e'] = kmeans.cluster_centers_[kmeans.labels_,2]
# tabela['c_d'] = kmeans.cluster_centers_[kmeans.labels_,3]
# tabela['c_sg'] = kmeans.cluster_centers_[kmeans.labels_,4]
tabela

c:\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG,cluster
0,FOR,38,10,11,17,34,44,41,-10,0
1,CFC,38,7,10,21,31,54,31,-23,3
2,SPT,38,12,6,20,31,50,42,-19,0
3,FLA,38,21,8,9,68,48,71,20,1
4,SAN,38,14,12,12,52,51,54,1,2
5,GRE,38,14,17,7,53,40,59,13,4
6,BOT,38,5,12,21,31,61,27,-30,3
7,COR,38,13,12,13,45,45,51,0,2
8,GOI,38,9,10,19,41,63,37,-22,3
9,PAL,38,15,13,10,51,37,58,14,4


In [25]:
print(kmeans.labels_)

[0 3 0 1 2 4 3 2 3 4 4 1 2 0 2 1 2 1 1 0]


In [26]:
kmeans.cluster_centers_

array([[ 42.        ,  11.        ,   9.        ,  18.        ,
        -14.75      ],
       [ 67.8       ,  19.4       ,   9.6       ,   9.        ,
         19.2       ],
       [ 52.        ,  13.6       ,  11.2       ,  13.2       ,
          0.2       ],
       [ 31.66666667,   7.        ,  10.66666667,  20.33333333,
        -25.        ],
       [ 56.66666667,  14.        ,  14.66666667,   9.33333333,
         12.33333333]])

In [28]:
df_cluster = pd.DataFrame()
for cluster, colunas in enumerate(kmeans.cluster_centers_):
    print(cluster)
    print(colunas)
    new_row = pd.DataFrame({
        'cluster': [cluster], 
        '2-P': [colunas[0]],              
        '4-V': [colunas[1]],
        '5-E': [colunas[2]],
        '6-D': [colunas[3]],
        '9-SG': [colunas[4]],
    })
    df_cluster = pd.concat([df_cluster, new_row], ignore_index=True)
df_cluster

0
[ 42.    11.     9.    18.   -14.75]
1
[67.8 19.4  9.6  9.  19.2]
2
[52.  13.6 11.2 13.2  0.2]
3
[ 31.66666667   7.          10.66666667  20.33333333 -25.        ]
4
[56.66666667 14.         14.66666667  9.33333333 12.33333333]


,cluster,2-P,4-V,5-E,6-D,9-SG
0,0,42.000000,11.0,9.000000,18.000000,-14.750000
1,1,67.800000,19.4,9.600000,9.000000,19.200000
2,2,52.000000,13.6,11.200000,13.200000,0.200000
3,3,31.666667,7.0,10.666667,20.333333,-25.000000
4,4,56.666667,14.0,14.666667,9.333333,12.333333


In [29]:
df_cluster_sort = df_cluster.sort_values(by='2-P', ascending=False)
df_cluster_sort

,cluster,2-P,4-V,5-E,6-D,9-SG
1,1,67.800000,19.4,9.600000,9.000000,19.200000
4,4,56.666667,14.0,14.666667,9.333333,12.333333
2,2,52.000000,13.6,11.200000,13.200000,0.200000
0,0,42.000000,11.0,9.000000,18.000000,-14.750000
3,3,31.666667,7.0,10.666667,20.333333,-25.000000


In [30]:
df_cluster_sort['grupo'] = ['Titulo','Libertadores','Sul-Americana','limbo','Rebaixamento']
df_cluster_sort

,cluster,2-P,4-V,5-E,6-D,9-SG,grupo
1,1,67.800000,19.4,9.600000,9.000000,19.200000,Titulo
4,4,56.666667,14.0,14.666667,9.333333,12.333333,Libertadores
2,2,52.000000,13.6,11.200000,13.200000,0.200000,Sul-Americana
0,0,42.000000,11.0,9.000000,18.000000,-14.750000,limbo
3,3,31.666667,7.0,10.666667,20.333333,-25.000000,Rebaixamento


In [31]:
df_cluster_grupo = df_cluster_sort[['cluster','grupo']]
df_cluster_grupo

,cluster,grupo
1,1,Titulo
4,4,Libertadores
2,2,Sul-Americana
0,0,limbo
3,3,Rebaixamento


In [32]:
tabela

,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG,cluster
0,FOR,38,10,11,17,34,44,41,-10,0
1,CFC,38,7,10,21,31,54,31,-23,3
2,SPT,38,12,6,20,31,50,42,-19,0
3,FLA,38,21,8,9,68,48,71,20,1
4,SAN,38,14,12,12,52,51,54,1,2
5,GRE,38,14,17,7,53,40,59,13,4
6,BOT,38,5,12,21,31,61,27,-30,3
7,COR,38,13,12,13,45,45,51,0,2
8,GOI,38,9,10,19,41,63,37,-22,3
9,PAL,38,15,13,10,51,37,58,14,4


In [100]:
import matplotlib 

tabela_sort = tabela.sort_values(by=['2-P','4-V','9-SG'], ascending=False)
tabela_sort.style.background_gradient(cmap='Blues').format("{:.2f}").render()


AttributeError: 'Styler' object has no attribute 'render'